# Day 4 - Google Search grounding with the Gemini API

## API: Get set up

This section requires an API key with billing enabled. Start by installing and importing the Gemini API Python SDK.

In [ ]:
%pip install -q -U 'google-generativeai>=0.8.3'

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

### Explore available models

Search grounding is a tool available in the `-002` series of models. Find a model that supports it through the [`models.list`](https://ai.google.dev/api/models#method:-models.list) endpoint. You can also find more information about different model capabilities on [the models page](https://ai.google.dev/gemini-api/docs/models/gemini).

In [ ]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)

## Search without grounding

In [ ]:
# Ask for information without search grounding.
model = genai.GenerativeModel("models/gemini-1.5-flash-002")

response = model.generate_content("Which country won the world cup most recently?")
rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

## Use search grounding

In [ ]:
# And now re-run the same query with search grounding enabled.
model = genai.GenerativeModel(
    "gemini-1.5-flash-002",
    tools="google_search_retrieval")

response = model.generate_content("Which country won the world cup most recently?")
rc = response.candidates[0]

Markdown(rc.content.parts[0].text)

The most recent country to win the FIFA World Cup is Argentina, who won the tournament in 2022.[1][2][3][4][5] They defeated France 4-2 in a penalty shootout after the game was tied 3-3 after extra time. This was Argentina's third World Cup title.[1][4][5][6]
  
Search Source:l
* wikipedia.org
* thehighlightsapp.com
* careerpower.in
* nbcsports.com
* statista.com
* wikipedia.org
edia.org

### Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.

In [ ]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)

As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

In [ ]:
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    print(support)

In [ ]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())

### Dynamic grounding

In a context where you may not know in advance whether to enable search grounding or not, you can provide the model with a threshold over which it will use search grounding. This is helpful in conversational contexts, where not every turn of conversation requires search data to support a response.

If you know whether to enable Search for any given chat turn, you can provide the tool explicitly.


<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1VBx_R16kNWa8g7lpLxQPx_08sFtd7tcd"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> Open in AI Studio</a>
  </td>
</table>

In [ ]:
nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
chat = nosearch_model.start_chat()

# No search grounding.
r = chat.send_message("Hello friendly chatbot!")

# Enable search for just this turn.
r = chat.send_message(
    "Who took home the 2023 cricket world cup?", tools="google_search_retrieval"
)

Markdown(r.text)

In [ ]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

Or you can let the Gemini API calculate a likelihood that the response needs search backing, and define the threshold to use.

In [ ]:
# The dynamic retrieval score is a probability, so the threshold
# must also be bound by [0, 1].
search_config = {
    "dynamic_retrieval_config": {"mode": "MODE_DYNAMIC", "dynamic_threshold": 0.5}
}

maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", tools={"google_search_retrieval": search_config}
)

chat = maybe_search_model.start_chat()

r = chat.send_message("Hello friendly chatbot!")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"First turn: {score=}")

r = chat.send_message("Who took home the 2023 cricket world cup?")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Second turn: {score=}")
print()

display(Markdown(r.text))

In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)